In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Data_for_UCI_named.csv')

In [3]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [5]:
data.shape

(10000, 14)

In [6]:
data = data.drop('stab', axis=1)

In [7]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import recall_score, classification_report, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

In [9]:
features_df = data.drop('stabf', axis=1)
target_var = data['stabf']

x_train, x_test, y_train, y_test = train_test_split(features_df, target_var, test_size=0.2, random_state=1)

scaler = StandardScaler()

x_train_norm = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test_norm = pd.DataFrame(scaler.transform(x_test), columns = x_test.columns)

In [10]:
rfc = RandomForestClassifier(random_state = 1)
rfc.fit(x_train_norm, y_train)
pred_1 = rfc.predict(x_test_norm)
accuracy = accuracy_score(y_test, pred_1)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.929


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [11]:
xgbc = XGBClassifier()
xgbc.fit(x_train_norm, y_train)
pred_2 = xgbc.predict(x_test_norm)
accuracy = accuracy_score(y_test, pred_2)
print('Accuracy: {}'.format(round(accuracy, 4)))

Accuracy: 0.9195


In [13]:
lgbmc = LGBMClassifier(random_state = 1)
lgbmc.fit(x_train_norm, y_train)
pred_3 = lgbmc.predict(x_test_norm)
accuracy = accuracy_score(y_test, pred_3)
print('Accuracy: {}'.format(round(accuracy, 4)))

Accuracy: 0.9395


In [15]:
etc = ExtraTreesClassifier(random_state = 1)
etc.fit(x_train_norm, y_train)
pred_4 = etc.predict(x_test_norm)
accuracy = accuracy_score(y_test, pred_4)

In [14]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

rsv = RandomizedSearchCV(etc, hyperparameter_grid, cv=5, n_iter = 10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)
search = rsv.fit(x_train_norm, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [16]:
search.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [17]:
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.928


In [18]:
etc = ExtraTreesClassifier(**search.best_params_, random_state = 1)
etc.fit(x_train_norm, y_train)
pred_5 = etc.predict(x_test_norm)
accuracy = accuracy_score(y_test, pred_5)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.927


In [20]:
feature_imp = pd.Series(etc.feature_importances_,  x_train_norm.columns).sort_values()
feature_imp

p1      0.003683
p4      0.004962
p2      0.005337
p3      0.005429
g1      0.102562
g2      0.107578
g4      0.109541
g3      0.113063
tau3    0.134680
tau4    0.135417
tau1    0.137240
tau2    0.140508
dtype: float64